## Analyzing borrowers’ risk of defaulting

My project is to prepare a report for a bank’s loan division. I’ll find out if a customer’s marital status and number of children has an impact on whether they will default on a loan. The bank already has some data on customers’ credit worthiness.

In [51]:
import pandas as pd
import numpy as np
import nltk
import math
from nltk.stem import WordNetLemmatizer
missing_values = ["n/a", "na", "--", "", " "]
customer_data = pd.read_csv(r'/datasets/credit_scoring_eng.csv', na_values=missing_values)
wordnet_lemma = WordNetLemmatizer()

customer_data.info()
customer_data.head(10)
customer_data.tail(10)
customer_data.describe()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       19351 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        19351 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


,children,days_employed,dob_years,education_id,family_status_id,debt,total_income
count,21525.000000,19351.000000,21525.000000,21525.000000,21525.000000,21525.000000,19351.000000
mean,0.538908,63046.497661,43.293380,0.817236,0.972544,0.080883,26787.568355
std,1.381587,140827.311974,12.574584,0.548138,1.420324,0.272661,16475.450632
min,-1.000000,-18388.949901,0.000000,0.000000,0.000000,0.000000,3306.762000
25%,0.000000,-2747.423625,33.000000,1.000000,0.000000,0.000000,16488.504500
50%,0.000000,-1203.369529,42.000000,1.000000,0.000000,0.000000,23202.870000
75%,1.000000,-291.095954,53.000000,1.000000,1.000000,0.000000,32549.611000
max,20.000000,401755.400475,75.000000,4.000000,4.000000,1.000000,362496.645000


### Conclusion

Data Potential Problems

1. null/nan values in 'days_employed' , 'total_income' 
2. days_employed - must be positive
3. dob_years - must be positive
4. education_id - must be corelated to the options
5. family_status - all lowercaps
6. family_status_id  - make sure everything is correlateed to the family status 
7. eduction - make everything in lowercaps
8. income type - all lowercaps
9. debt - make sure we have only 0 or 1
10. total income - make sure positive
11. purpose - make all lowercaps, then lemmatizing 

### Processing missing values

In [52]:
#Counting how many days emplyed rows are missing
print((customer_data.loc[customer_data['days_employed'].isnull()==True]).count())
#There are 2174 missing values for days_employed but also for total_income

#Turn days_employed to positive and then fill missing values
customer_data['days_employed'] = customer_data['days_employed'].abs()
customer_data['days_employed'].fillna(customer_data['days_employed'].median(), inplace=True)
customer_data['total_income'].fillna(customer_data['total_income'].median(), inplace=True)
customer_data.info()


#check how many negative days employed there are - 
print(customer_data.loc[0 > customer_data['days_employed']]) 
#result is more than 18,000 we cant drop them. therefore it is probably a human mistake and we will change the data to positive numbers

#Turning days employed into years
customer_data['days_employed'] = (customer_data['days_employed']/365)
days_employed_median = customer_data['days_employed'].median()
print(customer_data.loc[customer_data['days_employed'] > customer_data['dob_years']].count()) #- check how many days_employed don't make sense

#Change the values that does not make sense
customer_data.loc[(customer_data['days_employed'] > customer_data['dob_years']), 'days_employed'] = days_employed_median
print(customer_data.head())

#dob_years - must be positive
#There are 101 people that has 0 as their age, some of them even has children!
customer_data['dob_years'].value_counts()  

customer_data.loc[customer_data['dob_years'] == 0, 'dob_years'] = customer_data['dob_years'].median()
customer_data['dob_years'].value_counts()

children            2174
days_employed          0
dob_years           2174
education           2174
education_id        2174
family_status       2174
family_status_id    2174
gender              2174
income_type         2174
debt                2174
total_income           0
purpose             2174
dtype: int64
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       21525 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        21525 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB
Empty DataFrame

42.0    698
35.0    617
40.0    609
41.0    607
34.0    603
38.0    598
33.0    581
39.0    573
31.0    560
36.0    555
44.0    547
29.0    545
30.0    540
48.0    538
37.0    537
50.0    514
43.0    513
32.0    510
49.0    508
28.0    503
45.0    497
27.0    493
56.0    487
52.0    484
47.0    480
54.0    479
46.0    475
58.0    461
57.0    460
53.0    459
51.0    448
59.0    444
55.0    443
26.0    408
60.0    377
25.0    357
61.0    355
62.0    352
63.0    269
64.0    265
24.0    264
23.0    254
65.0    194
66.0    183
22.0    183
67.0    167
21.0    111
68.0     99
69.0     85
70.0     65
71.0     58
20.0     51
72.0     33
19.0     14
73.0      8
74.0      6
75.0      1
Name: dob_years, dtype: int64

### Conclusion

All the people that had missing values in the 'days_employed' column also had missing values in the 'total_income' column.
In addition some people in dob_years had 0 as their age.

3 types of missing values were detected:
1. days_employed - float (will be changed into int)
2. total_income - float (will be changed into int)
3. dob_years - int (some values were 0)


Possible reason:
To calculate the total income the algorithm uses the days_employed column. Becuase the last had no data, it was impossible to calculate the total_income.

How Blanks are filed:
I filled the blanks using the median() method. I decided to use the median over the mean in order to get more accurate results as the mean result is more affected by very small/big numbers

In addition I took care of the negative/impossible values in days employed. 

### Data type replacement

In [53]:
# columns needed to be replaced: 'days_employed','total_income'
try:
    customer_data['days_employed'] = customer_data['days_employed'].astype('int')
    customer_data['total_income'] = customer_data['total_income'].astype('int')
except:
    print('There has been an error in the conversion of days_employed and total_income columns, please check the data types or the code')


### Conclusion

Which method is used to change the data:

I used the astype() method in order to change the data from float type to int type. I decided to use that method because to_numeric() is used when we want to convert String(or object in Pandas) into a numeric type, but in this exmaple we already have numeric types and we want to change one type to another.

### Processing duplicates

In [54]:
#Notes - 
#customer_data['education'].value_counts() - need to make lower
#customer_data['family_status'].value_counts() - No need for lower
#customer_data['gender'].value_counts() - No need for lower
#customer_data['income_type'].value_counts() - No need for lower
#customer_data['purpose'].value_counts() - Need lower and stemming & lemmitization

#Taking Care of Case Sensitive Duplicates
try:
    customer_data['education'] = customer_data['education'].str.lower()
except:
    print('Something went wrong in changing the text to lower caps')
print(customer_data['education'].value_counts())


#Taking Care of Duplicates with Lemmatizing
#First we will tokenize the purpose words, after that we will lemmatize them and then get the stemms

try:
    customer_data['purpose'] = customer_data['purpose'].str.lower()
    purpose_lemmatizer = WordNetLemmatizer()
    lemmas_token = []
    lemmas_after_lem = []
except:
    print('Problem with making lower caps on purpose column')

secondary education    15233
bachelor's degree       5260
some college             744
primary education        282
graduate degree            6
Name: education, dtype: int64


In [55]:
#Tokenizing the purposes
try:   
    for word in customer_data['purpose']:
        lemmas_token.append(nltk.word_tokenize(word))
except:
    print('Problem with Tokenization or word already Tokenized')
    
#Lemmatizing the purposes
try:
    for element in lemmas_token:
        lemmas_after_lem.append([purpose_lemmatizer.lemmatize(w, pos = 'n') for w in element])
except:
    print('Problem with lemmatization')

#Updating the purpose column to lemmatized and seperated words
try:
    customer_data['purpose'] = lemmas_after_lem
    print(customer_data['purpose'])
except:
    print('problem with changing the values of purpose - maybe changed before')


0                           [purchase, of, the, house]
1                                      [car, purchase]
2                           [purchase, of, the, house]
3                           [supplementary, education]
4                               [to, have, a, wedding]
5                           [purchase, of, the, house]
6                               [housing, transaction]
7                                          [education]
8                                 [having, a, wedding]
9          [purchase, of, the, house, for, my, family]
10                                 [buy, real, estate]
11                     [buy, commercial, real, estate]
12                              [to, have, a, wedding]
13                                     [car, purchase]
14                    [buy, residential, real, estate]
15                   [construction, of, own, property]
16                                          [property]
17                             [building, a, property]
18        

In [56]:
#A function that returns a simple purpose category by using the lemmatized words
def change_purpose(data):
    try:
        if 'car' in data:
            return 'car'
        elif 'education' in data or 'university' in data or 'educated' in data:
            return 'education'
        elif 'wedding' in data or 'ceremony' in data:
            return 'wedding'
        elif 'real' in data or 'estate' in data or 'house' in data or 'property' in data or 'residential' in data or 'housing' in data:
            return 'housing'
        else:
            return 'other'
    except:
        print('problem with change_purpose function - maybe already been used')

print(customer_data['purpose'].value_counts())
customer_data['purpose_clean'] = customer_data['purpose'].apply(change_purpose)
print(customer_data['purpose_clean'].value_counts())

[car]                                            973
[wedding, ceremony]                              797
[having, a, wedding]                             777
[to, have, a, wedding]                           774
[real, estate, transaction]                      676
[buy, commercial, real, estate]                  664
[housing, transaction]                           653
[buying, property, for, renting, out]            653
[transaction, with, commercial, real, estate]    651
[purchase, of, the, house]                       647
[housing]                                        647
[purchase, of, the, house, for, my, family]      641
[construction, of, own, property]                635
[property]                                       634
[transaction, with, my, real, estate]            630
[building, a, real, estate]                      626
[buy, real, estate]                              624
[building, a, property]                          620
[purchase, of, my, own, house]                

In [57]:
#family_status - all lowercaps - alreday done
customer_data['family_status'].value_counts()

#family_status_id  - make sure everything is correlateed to the family status 
customer_data['family_status_id'].value_counts()

print(customer_data.loc[customer_data['family_status_id'] == 0]['family_status'].value_counts()) 
print(customer_data.loc[customer_data['family_status_id'] == 1]['family_status'].value_counts()) 
print(customer_data.loc[customer_data['family_status_id'] == 2]['family_status'].value_counts()) 
print(customer_data.loc[customer_data['family_status_id'] == 3]['family_status'].value_counts()) 
print(customer_data.loc[customer_data['family_status_id'] == 4]['family_status'].value_counts()) 

# 0 -married, 1- civil partnership, 2- widow / widower, 3- divorced, 4 -unmarried

print(len(customer_data.loc[((customer_data['family_status_id']==0) & (customer_data['family_status'] != 'married'))]))
print(len(customer_data.loc[((customer_data['family_status_id']==1) & (customer_data['family_status'] != 'civil partnership'))]))
print(len(customer_data.loc[((customer_data['family_status_id']==2) & (customer_data['family_status'] != 'widow / widower'))]))
print(len(customer_data.loc[((customer_data['family_status_id']==3) & (customer_data['family_status'] != 'divorced'))]))
print(len(customer_data.loc[((customer_data['family_status_id']==4) & (customer_data['family_status'] != 'unmarried'))]))

married    12380
Name: family_status, dtype: int64
civil partnership    4177
Name: family_status, dtype: int64
widow / widower    960
Name: family_status, dtype: int64
divorced    1195
Name: family_status, dtype: int64
unmarried    2813
Name: family_status, dtype: int64
0
0
0
0
0


In [58]:
#income type - all lowercaps - No need
customer_data['income_type'].value_counts()

#debt - make sure we have only 0 or 1 
customer_data['debt'].value_counts()

#total income - make sure positive
customer_data[customer_data['total_income']<0]

#check number of kids 
print(customer_data['children'].value_counts())

#Its not probable that there are 76 people with 20 kids or 47 with -1 kids.
#I assume this is a human error and 20 means 2 and -1 means 1
customer_data['children'] = customer_data['children'].abs()
customer_data.loc[customer_data['children'] == 20, 'children'] = 2
print(customer_data['children'].value_counts())

 0     14149
 1      4818
 2      2055
 3       330
 20       76
-1        47
 4        41
 5         9
Name: children, dtype: int64
0    14149
1     4865
2     2131
3      330
4       41
5        9
Name: children, dtype: int64


### Conclusion

Duplicates - 

in order to find duplicates i used the value_count method (there are some parts where it has been deleted after finding the duplicates). In order to take care of them i used the following methods: str.lower(), boolean indexing, boolean indexing with lemmatization.

The reason for duplicates in education and purpose is human error. when people get the chance to write by themself anyone writes differently which is leading to errors.

### Categorizing Data

In [59]:
customer_data.set_axis(['children', 'years_employed', 'age', 'education', 'education_id', 'family_status', 'family_status_id', 'gender', 'income_type', 'debt', 'total_income', 'purpose', 'purpose_clean'],  axis = 'columns',inplace=True)
customer_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 13 columns):
children            21525 non-null int64
years_employed      21525 non-null int32
age                 21525 non-null float64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        21525 non-null int32
purpose             21525 non-null object
purpose_clean       21525 non-null object
dtypes: float64(1), int32(2), int64(4), object(6)
memory usage: 2.0+ MB


In [60]:
#Categorizing clients by age
def age_group(age):
    try:
        if age<18:
            return 'children'
        elif 18<=age<65:
            return 'adult'
        else:
            return 'senior'
    except:
        print('There has been an error with the age group - please check the values of age column/function')

#Categorizing num of kids because 3 and above have small numbers
def children_group(num_of_kids):
    try:
        if num_of_kids == 0:
            return 'no kids'
        elif num_of_kids ==1:
            return 'one kid'
        elif num_of_kids ==2:
            return 'two kids'
        else:
            return 'three or more'
    except:
        print('Problem with children group - please check the data/function')

#Categorizing clients by salary to wealth groups
def wealth_group(salary):
    try:
        if salary<=17247:
            return 'low'
        elif 17247<salary<=23202:
            return 'average'
        elif 23202<salary<=31286:
            return 'above average'
        else:
            return 'high'
    except:
        print('There has been an error with wealth group function please check the data/function')
        
customer_data['age_group'] = customer_data['age'].apply(age_group)
customer_data['child_group'] = customer_data['children'].apply(children_group)
customer_data['wealth_group'] = customer_data['total_income'].apply(wealth_group)

customer_data['children'].value_counts()
customer_data['child_group'].value_counts()
print(customer_data.describe())

           children  years_employed           age  education_id  \
count  21525.000000    21525.000000  21525.000000  21525.000000   
mean       0.479721        5.961440     43.490453      0.817236   
std        0.755528        5.423352     12.218595      0.548138   
min        0.000000        0.000000     19.000000      0.000000   
25%        0.000000        2.000000     34.000000      1.000000   
50%        0.000000        6.000000     42.000000      1.000000   
75%        1.000000        6.000000     53.000000      1.000000   
max        5.000000       50.000000     75.000000      4.000000   

       family_status_id          debt   total_income  
count      21525.000000  21525.000000   21525.000000  
mean           0.972544      0.080883   26424.983229  
std            1.420324      0.272661   15658.580327  
min            0.000000      0.000000    3306.000000  
25%            0.000000      0.000000   17247.000000  
50%            0.000000      0.000000   23202.000000  
75%        

### Conclusion

The data has been categorized to the following categories:
1. Age Group - In order to able deduction from the age data
2. Child Group - In order to avoid insights from not enough data
3. Wealth Group - In order to be able to inffer insights from the total income data - the groups were made using decribe() method which shows the 25,50,75 precentage of the value total_income

### Step 3. Answer these questions

- Is there a relation between having kids and repaying a loan on time?

In [61]:
customer_data.head()

,children,years_employed,age,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,purpose_clean,age_group,child_group,wealth_group
0,1,23,42.0,bachelor's degree,0,married,0,F,employee,0,40620,"[purchase, of, the, house]",housing,adult,one kid,high
1,1,11,36.0,secondary education,1,married,0,F,employee,0,17932,"[car, purchase]",car,adult,one kid,average
2,0,15,33.0,secondary education,1,married,0,M,employee,0,23341,"[purchase, of, the, house]",housing,adult,no kids,above average
3,3,11,32.0,secondary education,1,married,0,M,employee,0,42820,"[supplementary, education]",education,adult,three or more,high
4,0,6,53.0,secondary education,1,civil partnership,1,F,retiree,0,25378,"[to, have, a, wedding]",wedding,adult,no kids,above average


In [76]:
num_ppl_default = len(customer_data.loc[customer_data['debt'] == 1])
num_ppl = len(customer_data['debt'])

customer_data.pivot_table(index = 'child_group', values = 'debt').sort_values('debt',ascending=False).round(4)*100

,debt
child_group,
two kids,9.48
one kid,9.15
three or more,8.16
no kids,7.51


### Conclusion

We can see that there is a relation between te number of kids a person have and the chance that they default. 
Someone with no kids will have less change to defult.
Chances to default in order from the most chances to the least:

1. 2 kids - 9.48%
2. 1 kids - 9.15%
3. 3 kids - 8.16%
4. 0 kids - 7.51%

- Is there a relation between marital status and repaying a loan on time?

In [73]:
customer_data.pivot_table(index='family_status', values='debt', aggfunc='mean').sort_values('debt',ascending=False).round(4)*100

,debt
family_status,
unmarried,9.74
civil partnership,9.29
married,7.52
divorced,7.11
widow / widower,6.56


### Conclusion

There is a connection between the martial status and repaying the loan.
Chances of repaying a loan -  from highest chance to default to the lowest:
1. Unmarried - 9.74%
2. Civil Partnership - 9.29% 
3. Married - 7.52%
4. Divorced - 7.11%
5. Widows - 6.56%

- Is there a relation between income level and repaying a loan on time?

In [74]:
customer_data.pivot_table(index='wealth_group', values='debt', aggfunc='mean').sort_values('debt',ascending=False).round(4)*100

,debt
wealth_group,
above average,8.87
average,8.46
low,7.93
high,7.17


### Conclusion

There are relation between the chance of default on a loan and the income level.
The highest a lowest income level has the smaller chances to default.
Chances to default on a loan from highest to lowest - 
1. Above average income - 8.87%
2. Average income - 8.46%
3. Low income - 7.93%
4. High income - 7.17%

- How do different loan purposes affect on-time repayment of the loan?

In [75]:
#Add customer id
counts = []

for i in range(len(customer_data)):
    counts.append(i)

customer_data['customer_id'] = counts
customer_data.head()

#Get default chance by purpose
customer_data.pivot_table(index='purpose_clean', values='debt', aggfunc='mean').sort_values('debt',ascending=False).round(4)*100

,debt
purpose_clean,
car,9.34
education,9.20
wedding,7.92
housing,7.21


### Conclusion

There are relations between the purpose of a loan and the chance to default on it.
Chances to default on a loan from the highest to the lowest:
1. Car loan - 9.34%
2. Education loan - 9.20%
3. Wedding loan - 7.92%
4. House loan - 7.21%

### Step 4. General conclusion

In [67]:
#Adding a column made of the average score of all the parameters we checked

#Insert chance to defalut in precentage by kids
def kids_precentage(kids_ctgory):
    try:
        if kids_ctgory == 'no kids':
            return 7.51
        elif kids_ctgory == 'one kid':
            return 9.15
        elif kids_ctgory == 'two kids':
            return 9.48
        else:
            return 8.16
    except:
        print('Kids precentage function error - please check the data/function')

#Insert chance to defalut in precentage by martial status
def marital_precentage(fam_stat_id):
    try:
        if fam_stat_id == 0:
            return 7.52
        elif fam_stat_id == 1:
            return 9.29
        elif fam_stat_id == 2:
            return 6.56
        elif fam_stat_id == 3:
            return 7.11
        else:
            return 9.74
    except:
        print('Error in martial precentage function - please check the data/function')

#Insert chance to defalut in precentage by income
def income_precentage(income):
    try:
        if income == 'low':
            return 7.93
        elif income == 'average':
            return 8.46
        elif income =='above average':
            return 8.87
        else:
            return 7.17
    except:
        print('Error in income precentage function - please check the data/function')

#Insert chance to defalut in precentage by purpose
def purpose_precentage(prpse_cln):
    try: 
        if prpse_cln == 'housing':
            return 7.21
        elif prpse_cln == 'car':
            return 9.34
        elif prpse_cln == 'wedding':
            return 7.92
        else:
            return 7.21
    except:
        print('Error in purpose precentage function - please check the data/function')

customer_data['children_default_precentage'] = customer_data['child_group'].apply(kids_precentage)
customer_data['family_status_default_precentage'] = customer_data['family_status_id'].apply(marital_precentage)
customer_data['income_default_precentage'] = customer_data['wealth_group'].apply(income_precentage)
customer_data['purpose_default_precentage'] = customer_data['purpose_clean'].apply(purpose_precentage)
customer_data['Chance_in_precentage_to_default'] = (customer_data['children_default_precentage'] + customer_data['family_status_default_precentage'] + customer_data['income_default_precentage'] + customer_data['purpose_default_precentage'])/4

customer_data.pivot_table(index='customer_id', values='Chance_in_precentage_to_default', margins=True).round(2)

,Chance_in_precentage_to_default
customer_id,
0,7.76
1,8.62
2,7.78
3,7.52
4,8.40
5,7.80
6,7.35
7,7.68
8,8.65


The General conclusion is that there is a relationship between the number of children, family status, income level, and purpose and the chances to repay the loan. 
I add another column (Chance_in_precentage_to_default) for your convenience with the chances of each customer to default on the loan.

The most likely to repay client the bank can have is - Widow with 0 kids, high salary that takes a loan for a house
The most likely to default client the bank can have is - unmarried  with 2 kids, above average salary that takes a loan for a car

Statisticas in % to default on a loan:


In [68]:
#by purpose

customer_data.pivot_table(index='purpose_clean', values='debt', aggfunc='mean').sort_values('debt',ascending=False).round(4)*100

,debt
purpose_clean,
car,9.34
education,9.20
wedding,7.92
housing,7.21


In [69]:
#by income

customer_data.pivot_table(index='wealth_group', values='debt', aggfunc='mean').sort_values('debt',ascending=False).round(4)*100

,debt
wealth_group,
above average,8.87
average,8.46
low,7.93
high,7.17


In [70]:
#by maritial status

customer_data.pivot_table(index='family_status', values='debt', aggfunc='mean').sort_values('debt',ascending=False).round(4)*100

,debt
family_status,
unmarried,9.74
civil partnership,9.29
married,7.52
divorced,7.11
widow / widower,6.56


In [71]:
#by kids

customer_data.pivot_table(index = 'child_group', values = 'debt').sort_values('debt',ascending=False).round(4)*100

,debt
child_group,
two kids,9.48
one kid,9.15
three or more,8.16
no kids,7.51
